**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Jun 10 07:50:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**1) Clone the Darknet**



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13652, done.
remote: Total 13652 (delta 0), reused 0 (delta 0), pack-reused 13652
Receiving objects: 100% (13652/13652), 12.24 MiB | 11.67 MiB/s, done.
Resolving deltas: 100% (9312/9312), done.


**2) Compile Darknet using Nvidia GPU**


In [5]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [0]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov_3"

In [0]:
!echo "plate" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [10]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-06-10 07:54:04--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   109KB/s    in 34m 42s 

2020-06-10 08:28:48 (76.2 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [12]:
!unzip '/content/gdrive/My Drive/Cars.zip' -d data/obj

Archive:  /content/gdrive/My Drive/Cars.zip
  inflating: data/obj/0.jpg          
  inflating: data/obj/0.txt          
  inflating: data/obj/1.jpg          
  inflating: data/obj/1.txt          
  inflating: data/obj/2.jpg          
  inflating: data/obj/2.txt          
  inflating: data/obj/3.jpg          
  inflating: data/obj/3.txt          
  inflating: data/obj/4.jpg          
  inflating: data/obj/4.txt          
  inflating: data/obj/5.jpg          
  inflating: data/obj/5.txt          
  inflating: data/obj/6.jpg          
  inflating: data/obj/6.txt          
  inflating: data/obj/7.jpg          
  inflating: data/obj/7.txt          
  inflating: data/obj/8.jpg          
  inflating: data/obj/8.txt          
  inflating: data/obj/9.jpg          
  inflating: data/obj/9.txt          
  inflating: data/obj/10.jpg         
  inflating: data/obj/10.txt         
  inflating: data/obj/11.jpg         
  inflating: data/obj/11.txt         
  inflating: data/obj/12.jpg         
  infl

In [13]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.442 0.455223880597014 0.862 0.686567164179104
['0', '0.442', '0.455223880597014', '0.862', '0.686567164179104']
0 data/obj/173.txt
0 0.442 0.455223880597014 0.862 0.686567164179104
0 0.37058823529411705 0.386196769456681 0.64 0.47283406754772306
['0', '0.37058823529411705', '0.386196769456681', '0.64', '0.47283406754772306']
1 data/obj/35.txt
0 0.37058823529411705 0.386196769456681 0.64 0.47283406754772306
0 0.330188679245283 0.5983935742971881 0.764150943396226 0.8835341365461841
['0', '0.330188679245283', '0.5983935742971881', '0.764150943396226', '0.8835341365461841']
2 data/obj/230.txt
0 0.330188679245283 0.5983935742971881 0.764150943396226 0.8835341365461841
0 0.285092491838955 0.35714285714285704 0.80739934711643 0.7387218045112781
['0', '0.285092491838955', '0.35714285714285704', '0.80739934711643', '0.7387218045112781']
3 data/obj/19.txt
0 0.285092491838955 0.35714285714285704 0.80739934711643 0.7387218045112781
0 0.276 0.391428571428571 0.756 0.5942857142857141
['0', '0.2

In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/78.jpg', 'data/obj/154.jpg', 'data/obj/52.jpg', 'data/obj/166.jpg', 'data/obj/81.jpg', 'data/obj/46.jpg', 'data/obj/44.jpg', 'data/obj/133.jpg', 'data/obj/7.jpg', 'data/obj/86.jpg', 'data/obj/130.jpg', 'data/obj/67.jpg', 'data/obj/181.jpg', 'data/obj/192.jpg', 'data/obj/139.jpg', 'data/obj/5.jpg', 'data/obj/195.jpg', 'data/obj/231.jpg', 'data/obj/184.jpg', 'data/obj/112.jpg', 'data/obj/105.jpg', 'data/obj/87.jpg', 'data/obj/174.jpg', 'data/obj/230.jpg', 'data/obj/71.jpg', 'data/obj/182.jpg', 'data/obj/223.jpg', 'data/obj/0.jpg', 'data/obj/58.jpg', 'data/obj/183.jpg', 'data/obj/4.jpg', 'data/obj/173.jpg', 'data/obj/65.jpg', 'data/obj/95.jpg', 'data/obj/125.jpg', 'data/obj/224.jpg', 'data/obj/229.jpg', 'data/obj/171.jpg', 'data/obj/110.jpg', 'data/obj/11.jpg', 'data/obj/59.jpg', 'data/obj/202.jpg', 'data/obj/222.jpg', 'data/obj/203.jpg', 'data/obj/41.jpg', 'data/obj/210.jpg', 'data/obj/36.jpg', 'data/obj/2.jpg', 'data/obj/83.jpg', 'data/obj/214.jpg', 'data/obj/162.jpg', 'data/

In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [16]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.853623, GIOU: 0.853043), Class: 0.993354, Obj: 0.350655, No Obj: 0.001340, .5R: 1.000000, .75R: 1.000000, count: 4, class_loss = 0.588956, iou_loss = 0.183409, total_loss = 0.772365 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000007, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000002, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 72745, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.773425, GIOU: 0.760741), Class: 0.982174, Obj: 0.396901, No Obj: 